<a href="https://colab.research.google.com/github/0xLighty/Natural-Language-Processing/blob/main/hw4_tast2_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2
Import BERT for classification model which is already fine tuned and apply it on any text classification dataset such as the twitter dataset

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import get_scheduler
import evaluate
from tqdm import tqdm
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [3]:
train = load_dataset('tweet_eval', 'emotion', split='train[:2000]')
val = load_dataset('tweet_eval', 'emotion', split='validation[:200]')

In [4]:
type(train)

datasets.arrow_dataset.Dataset

In [5]:
train

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [6]:
train['text'][:10]

["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs",
 "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu",
 "Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight",
 "it's pretty depressing when u hit pan on ur favourite highlighter",
 '@user but your pussy was weak from what I heard so stfu up to me bitch . You got to threaten him that your pregnant .',
 'Making that yearly transition from excited and hopeful college returner to sick and exhausted pessimist. #college',
 'Tiller and breezy should do a collab album. Rapping and singing prolly be fire',
 '@user broadband is shocking regretting signing up now #angry #shouldofgonewithvirgin',
 '@user Look at those teef! #growl']

In [7]:
train['label'][:10]

[2, 0, 1, 0, 3, 0, 3, 1, 0, 0]

In [8]:
# there are four classes/labels/emotions
set(train['label'])

{0, 1, 2, 3}

In [9]:
# from transformers import BertForQuestionAnswering, BertForSequenceClassification,BertTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline #, AdamW, BertTokenizerFast, Trainer, TrainingArguments

In [10]:
tokenizer = AutoTokenizer.from_pretrained('philschmid/BERT-tweet-eval-emotion')
model = AutoModelForSequenceClassification.from_pretrained('philschmid/BERT-tweet-eval-emotion')
classifier = pipeline('text-classification', tokenizer=tokenizer, model=model)

In [11]:
# taking random text and it's relevant class/emotion/label
random_text = []
random_num = np.random.randint(0,len(train))
random_text.append(train['text'][random_num])
cls_label = train['label'][random_num]

In [12]:
# encoding random text with max_length of 512
input_ids = tokenizer.encode(random_text,is_split_into_words=True, max_length=512,truncation=True)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 42 tokens.


In [13]:
# visualizing the tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
@            137
user       4,795
King       1,624
never      1,309
once       1,517
spoke      2,910
out        1,149
about      1,164
how        1,293
the        1,103
Left       8,123
crush     11,321
##es       1,279
#            108
Free       4,299
##S        1,708
##pe       3,186
##ech     11,252
in         1,107
publishing   5,550
world      1,362
.            119
\            165
n            183
\            165
n            183
#            108
Trump      8,499
#            108
horror     5,367
#            108
s            188
##ci       6,617
##fi       8,702
#            108
cc        14,402
##ot       3,329
#            108
p            185
##2        1,477
[SEP]        102


In [14]:
# Visualizing the number of token in text
sep_token_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_token_idx)

SEP token index:  41


In [15]:
num_seg_a = sep_token_idx + 1
print("Number of tokens in text: ", num_seg_a)

Number of tokens in text:  42


In [16]:
#creating the segment ids and making sure every input token has a segment id
segment_ids = [0]*num_seg_a 
assert len(segment_ids) == len(input_ids)

In [17]:
output = model(torch.tensor([input_ids]),  token_type_ids = torch.tensor([segment_ids]))

In [18]:
pred = int(torch.argmax(output.logits))

In [19]:
print("Predicted class : {} - Original Class : {}".format(pred, cls_label))

Predicted class : 0 - Original Class : 0
